# Interactive sliding Value-at-Risk with atoti

## A quick introduction

This jupyter notebook will demonstrate how atoti™ community edition can help to analyse and explore **Value-at-Risk** across observation windows. This tool can be particularly useful for clients looking to **recalibrate the period** of significant stress relevant to their portfolio, for instance, from the **Global Financial Crisis of 2008** to the **Coronavirus Market Crash of 2020**. 

Read more about the use case in this blog post: [Stressed VaR calibration](#).

We will focus on the step-by-step implementation of an analytical cube for the rolling VaR use case in atoti.


# Importing libraries

We will be using atoti and pandas today.

In [1]:
import atoti as tt
import pandas as pd

Welcome to atoti 0.4.0.20200519221045!

By using this community edition, you agree with the license available at https://www.atoti.io/eula.
Browse the official documentation at https://docs.atoti.io.
Join the community at https://www.atoti.io/register.

You can hide this message by setting the ATOTI_HIDE_EULA_MESSAGE environment variable to True.


Each PnL value in the returned vector corresponds to a scenario at the same index position.

# Sample data overview

For the purposes of this demonstration, we'll use a toy data model that can be adjusted for a real-life use case. The datastores that we'll need include:

- positions: books, stock tickers and positions market values in USD (long and short positions),
- historical returns: vectors of stock prices and daily changes, we will be multiplying market values by stock price changes to obtain a simulated historical P&L for the demonstration. The first two data stores can be replaced with full revaluation P&L vectors from a risk system.

# Creating an atoti session and loading data

In [2]:
# Starting atoti session
session = tt.create_session()

# Let's load historical data into a store from a csv file.
# Read more about the supported input data sources on the https://docs.atoti.io/.
# By setting array_sep, atoti will recognize the input data is in vectorized format.

market_data_store = session.read_csv(
    "stock_price_returns.csv",
    keys=["Stock"],
    store_name="StockData",
    array_sep=";"
)

market_data_store.head(10)

,Returns,Prices
Stock,,
AAPL,"doubleVector[3632]{0.0029431159209480917, ...}","doubleVector[3632]{9.271618843078613, ...}"
GOOG,"doubleVector[3632]{0.022999346863395997, ...}","doubleVector[3632]{221.78868103027344, ...}"
LUKOY,"doubleVector[3632]{0.0195919993640441, ...}","doubleVector[3632]{32.09617614746094, ...}"
MSFT,"doubleVector[3632]{0.004844138140399634, ...}","doubleVector[3632]{19.697484970092773, ...}"
OGZPY,"doubleVector[3632]{0.05288110091337339, ...}","doubleVector[3632]{7.355556964874268, ...}"
ROSYY,"doubleVector[3632]{0.023913277535070776, ...}","doubleVector[3632]{8.996960639953613, ...}"
^DJI,"doubleVector[3632]{0.003018253564988971, ...}","doubleVector[3632]{10880.150390625, ...}"
^GSPC,"doubleVector[3632]{0.0036726922525569528, ...}","doubleVector[3632]{1273.4599609375, ...}"


In [3]:
# Now let's load the positions.
positions_store = session.read_csv(
    "positions_snapshot.csv",
    keys=["book", "Stock"],
    store_name="Positions",
    types={"AmountUSD": tt.types.DOUBLE},
)

positions_store.head(3)

portfolio  AmountUSD
book           Stock                                
Indices        ^DJI   US stocks portfolio     1000.0
               ^GSPC  US stocks portfolio     1000.0
Technology CFD AAPL   US stocks portfolio     -500.0

In [4]:
positions_store.join(market_data_store)

# Creating an OLAP cube

In [5]:
cube = session.create_cube(positions_store, "Historical VaR Cube", mode="auto")
h = cube.hierarchies
l = cube.levels
m = cube.measures
cube

Let's put Books under Portfolios to create a multi-level hierarchy:

In [6]:
h["Portfolio Structure"] = {
    "Portfolio": l["portfolio"],
    "Book": l['book']
}

The default aggregation functions created at this step allow to display position's market values. I'm using the multi-level `Portfolio Structure` to expand data:

In [8]:
cube.visualize()

Install and enable the atoti JupyterLab extension to see this widget.

# New measure for the VaR vector

In our toy example we multiply the positions market values -  _AmountUSD_  - by the stock returns for each name to obtain the historical VaR vectors. This step is not necessary if PL vectors are delivered by a risk system. 

In [8]:
m["VaR Vector"] = tt.agg.sum(
    m["AmountUSD.SUM"] * m["Returns.VALUE"], scope=tt.scope.origin(l["Stock"])
)

We can now explore the VaR vectors at stock and book level.

In [10]:
cube.visualize()

Install and enable the atoti JupyterLab extension to see this widget.

# Enriching VaR vectors with date labels

So far, VaR vectors are not intuitive. Let's create a hierarchy that will allow to display P&L value by historical date, which is more user friendly.

In [11]:
calendar = pd.date_range(end = '2020-06-09', periods = 3632, freq="B")

In [12]:
cube.create_parameter_hierarchy(
    "Historical Dates", calendar, index_measure="Date Index",
)

In [13]:
# This new measure will pick an element from the VaR vector:
m["PnL at date"] = m["VaR Vector"][m["Date Index"]]
m["PnL at date"].formatter = "DOUBLE[#,##0.#]"
m["Price at date"] = m["Prices.VALUE"][m["Date Index"]]

Let's do some simple verification. In the following Pivot table (filtered for the 4 recent dates), we can see the PnL value for each date scenario.  

For each date, let's drill down to the books and stock. 

In [15]:
cube.visualize()

Install and enable the atoti JupyterLab extension to see this widget.

Or, for prices (this view is filtered for three most recent dates):

In [17]:
cube.visualize()

Install and enable the atoti JupyterLab extension to see this widget.

# Creating Sliding Windows

We want to compute VaRs for different observation windows to see which period would lead to the most conservative VaR for our current portfolio. Such a "worst" window will be a good candidate for the Stressed VaR calibration and will be a focus for further analysis and decomposition.

In [18]:
# Lets create a measure for the observation window length:
m['Lookback Window'] = 260

In [19]:
m["VaR Sub-Vector"] = m["VaR Vector"][
    m["Date Index"] - m['Lookback Window'] : m["Date Index"]
]

The Sub-Vector measure requires having historical dates in the view. 

Now for each desired scope, the `VaR Sub-Vector` measure will pick 260 most recent values from the `VaR vector`. The below view is filtered for the 3 most recent historical dates. This result is not for human consumption, but still showing it here for illustration purposes.

In [21]:
cube.visualize()

Install and enable the atoti JupyterLab extension to see this widget.

# Adding Sliding VaR measure

In this notebook, we'll apply a simple quantile function to the subvector. 

Please read about about **VaR-type estimators using kernel functions** in our future blog posts.

In [22]:
m["1d 99% VaR Sliding"] = tt.array.quantile(m["VaR Sub-Vector"], 0.01)

As it's based on the `VaR Sub-Vector` measure, it also requires the `Historical Dates` in the view:

As expected, VaR chart is step-wise and goes up and down every time extreme scenarios leave the observation window.

In [24]:
cube.visualize()

Install and enable the atoti JupyterLab extension to see this widget.

# Quantiles for stock returns

In [25]:
m["Returns Sub-Vector 260 window"] = m["Returns.VALUE"][
    m["Date Index"] - 260 : m["Date Index"]
]
m["Empirical 99% Percentile based on 260 days rolling window"] = tt.array.quantile(m["Returns Sub-Vector 260 window"], 0.01)

m["Returns Sub-Vector 130 window"] = m["Returns.VALUE"][
    m["Date Index"] - 130 : m["Date Index"]
]
m["Empirical 99% Percentile based on 130 days rolling window"] = tt.array.quantile(m["Returns Sub-Vector 130 window"], 0.01)

In [47]:
m['1d pct price change'] = m['Returns.VALUE'][m["Date Index"]]
m['1d pct price change'].formatter = "DOUBLE[#%]"

In [44]:
cube.visualize()

Install and enable the atoti JupyterLab extension to see this widget.

# Drill down to the worst scenarios

In [46]:
cube.visualize()

Install and enable the atoti JupyterLab extension to see this widget.

# What-If on different lookback setting

Now let's setup a what-if simulation and change the Lookback Window to shorter and longer lengths. 

In [29]:
lookback_windows = cube.setup_simulation(
    "Lookback Windows", replace=[m["Lookback Window"]], base_scenario="260 days"
).scenarios
lookback_windows["10 days"] = 10
lookback_windows["120 days"] = 120
lookback_windows["2600 days"] = 2600

Now the different `Lookback Windows` are available in the UI for analysis. As expected, the shorter the observation window, the faster VaR metric reacts to changes in market volatility.

In [31]:
cube.visualize()

Install and enable the atoti JupyterLab extension to see this widget.

# Worst VaR period

Now let's create a measure that will compute the worst VaR for any scope of data - for example, books or legal entities - that we want to have in the view.

In [48]:
# the measure should compute the worst VaR across sliding windows - doesn't work as desired unfortunately
m['Worst VaR'] = tt.agg.min(m["1d 99% VaR Sliding"], scope = tt.scope.origin(l['Historical Dates']))

In [34]:
cube.visualize()

Install and enable the atoti JupyterLab extension to see this widget.

And this measure will return the worst scenario:

In [ ]:
# the measure should compute the worst sliding window - doesn't work as desired unfortunately
m['Worst VaR Period'] = tt.agg.min_member(m["1d 99% VaR Sliding"], l['Historical Dates'])

In [39]:
cube.visualize()

Install and enable the atoti JupyterLab extension to see this widget.